In [1]:
import os

In [2]:
UNIMORPH_DIR = "/home/alexeysorokin/data/Data/UniMorph/"
UD_DIR = "/home/alexeysorokin/data/Data/UD2.3/"
UD_TEST_DIR = "/home/alexeysorokin/data/Data/UD2.3/"

In [3]:
good_languages, bad_languages = [], []
for folder in os.listdir(UD_DIR):
    language = folder[3:].split("-")[0].lower()
    if language in os.listdir(UNIMORPH_DIR):
        to_append = good_languages
    else:
        to_append = bad_languages
    to_append.append((folder, language))
len(good_languages), len(bad_languages)

(77, 52)

In [4]:
def count_common(first, second):
    count, correct_count, total = 0, 0, 0
    for word, word_data in first.items():
        word_tags = [tag for tag in word_data if tag.split(",")[0] in ["VERB", "NOUN", "ADJ"]]
        total += len(word_tags)
        if len(word_tags) > 0:
            if word in second:
                dict_tags = second[word]
                count += len(word_tags)
                correct_count += len([tag for tag in word_tags if tag in dict_tags])
    return count, correct_count, total

In [5]:
def count_with_guesser(guesser, data):
    answer = []
    count, correct_count = 0, 0
    for word, word_data in data.items():
        word_tags = [tag for tag in word_data if tag.split(",")[0] in ["VERB", "NOUN", "ADJ"]]
        if len(word_tags) > 0:
            dict_tags = guesser[word]
            if dict_tags is not None and len(dict_tags) > 0:
                count += len(word_tags)
                correct_count += len([tag for tag in word_tags if tag in dict_tags])
#             if any(isinstance(x, int) for x in dict_tags):
#                 print(word, dict_tags)
            answer.append((word, word_tags, guesser._unimorph_data[word], dict_tags))
    return count, correct_count, answer

In [6]:
%load_ext autoreload

In [7]:
%aimport neural_tagging.vectorizers
%aimport neural_tagging.suffix_guesser
from neural_tagging.vectorizers import read_unimorph_infile, read_ud_infile
from neural_tagging.vectorizers import *
from neural_tagging.suffix_guesser import SuffixGuesser

Using TensorFlow backend.


In [8]:
good_languages = [("UD_Belarusian-HSE", "belarusian")]

In [15]:
GUESSER_PARAMS = {"max_suffix_length": 8, "min_impurity": 1e-3, "symbols_to_remove": "́", "min_suffix_to_guess": 2}

In [17]:
%autoreload 1

for folder, language in sorted(good_languages):
    curr_folder = os.path.join(UD_DIR, folder)
    files = [x for x in os.listdir(curr_folder) if x.endswith(".conllu")]
    train_file, dev_file = None, None
    for infile in files:
        if "train" in infile:
            train_file = infile
        elif "dev" in infile:
            dev_file = infile
        elif "test" in infile:
            test_file = infile
    if test_file is None:
        if dev_file is not None:
            train_files, test_file = [train_file], dev_file
        else:
            continue
    else:
        train_files = [train_file, dev_file] if dev_file is not None else [train_file]
    unimorph_file = os.path.join(UNIMORPH_DIR, language)
    test_file = os.path.join(curr_folder, test_file)
    
    train_files = [os.path.join(curr_folder, infile) for infile in train_files]
    train_data = read_ud_infile(train_files)
    unimorph_data = read_unimorph_infile(unimorph_file, by_lemmas=True, to_list=True)
    dev_data = read_ud_infile(test_file)
    
    guesser_file = "neural_tagging/models/guessers/{}".format(language)
    if not os.path.exists(guesser_file):
        guesser = SuffixGuesser(**GUESSER_PARAMS).train(unimorph_data)
        guesser.save(guesser_file)
    vectorizer = MatchingVectorizer(verbose=0, guesser=guesser_file, guessed_weight=1.0,
                                    prob_threshold=0.75, pos_count_threshold=50,
                                    source_forms_to_add={"V": "V;INF"}).train(unimorph_file, train_files)
    
    common, corr_common, total = count_common(dev_data, train_data)
    uni_common, uni_corr_common, answer = count_with_guesser(vectorizer, dev_data)
    print(folder[3:], total, common, corr_common, uni_common, uni_corr_common, len(answer))
    with open("neural_tagging/dump/matches/{}.match".format(folder), "w", encoding="utf8") as fout:
        for word, word_tags, uni_tags, trans_tags in answer:
#             if any(x in trans_tags for x in word_tags):
#                 continue
            if trans_tags is None:
                trans_tags = []
            if uni_tags is None:
                uni_tags = []
            fout.write(word + "\n")
            for tag in word_tags:
                fout.write("{} {}\n".format(tag, tag in trans_tags))
            fout.write("-" * 50 + "\n")
            for tag in uni_tags:
                fout.write(tag + "\n")
            fout.write("-" * 50 + "\n")
            for tag in trans_tags:
                fout.write(tag + "\n")
            fout.write("\n")

Belarusian-HSE 495 184 148 381 260 479
